<a href="https://colab.research.google.com/github/elliot-drew/simpy-notebooks/blob/main/farm_shop_simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Farm shop - SimPy

Below I have put together a very simple simulation using SimPy.

Model is a shop that is supplied by a farm. Customers visit the shop and buy items. 

Its all fairly straightforward. The visits by customers have been modelled as a Poisson process as it they are discrete events and we can know the average number of these events a day - but the timing of them is random. An exponential distribution can be used as the probability distribution for the times between these events.

The variables for the model (e.g. mean rate of customers visiting shop a day, size of purchase, etc) I've pulled out of thin air. Used the `names` python library for the customer names as its more fun.

At the end I do a very simple and linear search for the minimum resupply order point that results in no customers waiting - the level of inventory where you order more items for the shop. If I was doing this for real, many more simulations/repeats would be necessary, and its likely I'd be trying to optimise for more than one variable... for that I might use something like the `optimize` functions of the scipy library.

I think one could fairly easily add more steps in the supply chain to this model: 

* potentially another shop that is supplied from the current shop, with its own customers
* multiple shops all being supplied by the farm. 
* The farm could be given some properties - how much it can produce, at what rate. 
* Customers could buy a random amount of items
* introduce different products - maybe each farm makes a different item?



In [ ]:
!pip install simpy
!pip install names

     |████████████████████████████████| 789 kB 4.9 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803698 sha256=ce6cc000775cd93971e4203d548957aae7c18ff911f3737a534550019c677b7f
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names


In [ ]:
import simpy
import numpy as np
import math
import names
from itertools import repeat

In [ ]:
# define some model parameters in a class so things are neat

class ModelParams:
  # model: Farm -> Shop -> Customer
  # Customers arrive at a given rate per day
  order_rate = 3
  # Customers buy X item per visit (for simplicity)
  order_size = 1
  # Farm resupplies Shop in lots of X items
  f_s_resup_size = 20
  # Farm needs to know X days in advance
  f_s_resup_adv = 5
  # Shop asks for resupply when inventory hits this level
  s_rep_ord_point = 1*5*5
  # time to simulate
  sim_time = 365




In [ ]:
# Inventory class
# instantiate and monitor the shop inventory

class Inventory:
  def __init__(self, env):
    # p is an instance of ModelParams
    self.env = env # simulation environment
    self.shop_inv = simpy.Container(env, init = ModelParams.s_rep_ord_point)
    self.monitor_shop = env.process(self.monitor_shop_inv(env)) # monitors inv
  
  def monitor_shop_inv(self, env):
    while True: # inf loop

      if(self.shop_inv.level <= ModelParams.s_rep_ord_point):
        # the shop inventory is low and reorder point hit
        print("{0}: Shop reorder point hit".format(self.env.now))
        yield self.env.timeout(ModelParams.f_s_resup_adv) # time till resup
        print("{0}: Shop resupplied ({1})".format(self.env.now, self.shop_inv.level))
        yield self.shop_inv.put(ModelParams.f_s_resup_size) # resup happens
      
      yield self.env.timeout(1) #tick tock


In [ ]:
class Stats:
  inv = None
  shop_waits = [] # waiting times for customers at shop 
  number_shop_custs = 0 # number of customers at the shop
  shop_cust_trajectory = list(repeat(0, ModelParams.sim_time)) 

In [ ]:
# ShopCustomer class
# has one action - buys 

class ShopCustomer(object):
  def __init__(self, env):
    self.env = env
    self.action = self.env.process(self.order_from_shop())
    self.name = names.get_full_name() # for fun
  
  def order_from_shop(self):
    # s is a Stats instance
    start_time = self.env.now
    day = math.floor(start_time)
    Stats.shop_cust_trajectory[day] += 1 # a customer has come in on day
    # customer places an order
    #print("{0}: {1} orders something from Shop".format(self.env.now, self.name))
    yield Stats.inv.shop_inv.get(ModelParams.order_size)
    # customer receives order
    #print("{0}: {1} receives order from Shop".format(self.env.now, self.name))
    # how long did they wait?
    wait_time = self.env.now - start_time
    print("{0} had to wait for {1} days.".format(self.name, wait_time))
    # store wait time in Stats class instance
    Stats.shop_waits.append(wait_time)

In [ ]:
# need to process that will generate customers for the shop

class ShopCustomerGen:
  def __init__(self, env, mean_rate):
    self.env = env
    self.action = self.env.process(self.new_customer_entrance())
    self.mean_rate = mean_rate
  
  def new_customer_entrance(self):
    rng = np.random.default_rng()
    while True:
      # time between customers pulled form exponential distribution - poisson
      # process
      scale = 1/self.mean_rate
      time_between_arrivals = rng.exponential(scale)
      yield self.env.timeout(time_between_arrivals)
      # make a new customer
      c = ShopCustomer(self.env)
      # increment count
      Stats.number_shop_custs += 1


In [ ]:
# everything is now set up. Make a function to run the model here.

def model():
  # set up the simpy environment
  environment = simpy.Environment()
  
  shop = ShopCustomerGen(environment, ModelParams.order_rate)
  Stats.inv = Inventory(environment)
  environment.run(until = ModelParams.sim_time)
  return Stats.shop_waits, Stats.number_shop_custs, Stats.shop_cust_trajectory


In [ ]:
waits, cust_no, cust_traj = model()

0: Shop reorder point hit
David Peacock had to wait for 0.0 days.
Francis Schrum had to wait for 0.0 days.
Angelo Petersen had to wait for 0.0 days.
Irene Collymore had to wait for 0.0 days.
Cameron Evans had to wait for 0.0 days.
Jessica Johnson had to wait for 0.0 days.
Patrick Hansen had to wait for 0.0 days.
Jonathan Jeffries had to wait for 0.0 days.
Rodney Smith had to wait for 0.0 days.
Andrew Anderson had to wait for 0.0 days.
Joyce Bon had to wait for 0.0 days.
5: Shop resupplied (14)
Anthony Bales had to wait for 0.0 days.
Jason Kennedy had to wait for 0.0 days.
Robert Murphy had to wait for 0.0 days.
Jordan Dyer had to wait for 0.0 days.
Michelle Marable had to wait for 0.0 days.
Sarah Mcmillan had to wait for 0.0 days.
Ruth Peachey had to wait for 0.0 days.
Nancy West had to wait for 0.0 days.
John Couch had to wait for 0.0 days.
8: Shop reorder point hit
Jessica Dufilho had to wait for 0.0 days.
Dianne Hernandez had to wait for 0.0 days.
Mary Olivarez had to wait for 0.0 d

In [ ]:
print("mean days waiting:", np.mean(waits))

mean days waiting: 0.0


In [ ]:
# optimise to find best ROP to minimise days spent waiting given 4 orders a day.

mean_wait = 99999 # daft high number to begin with
current_rop = 25


while mean_wait > 0:
  
  # change params
  # Customers arrive at a given rate per day
  ModelParams.order_rate = 3.2
  # Customers buy X item per visit (for simplicity)
  ModelParams.order_size = 1
  # Farm resupplies Shop in lots of X items
  ModelParams.f_s_resup_size = 20
  # Farm needs to know X days in advance
  ModelParams.f_s_resup_adv = 5
  # Shop asks for resupply when inventory hits this level
  ModelParams.s_rep_ord_point = current_rop
  # time to simulate
  ModelParams.sim_time = 365

  # reset Stats
  Stats.inv = None
  Stats.shop_waits = [] # waiting times for customers at shop 
  Stats.number_shop_custs = 0 # number of customers at the shop
  Stats.shop_cust_trajectory = list(repeat(0, ModelParams.sim_time)) 

  waits, cust_no, cust_traj = model()
  
  mean_wait = np.mean(waits)
  std_wait = np.std(waits)
  print("ROP: {2} MEAN WAIT: {0} +/- {1}".format(round(mean_wait,5), round(std_wait,5), current_rop))
  current_rop += 1


Streaming output truncated to the last 5000 lines.
Dennis Gallagher had to wait for 0.0 days.
Alvaro Mears had to wait for 0.0 days.
Brandon Barker had to wait for 0.0 days.
Phyllis Jenkins had to wait for 0.0 days.
29: Shop resupplied (7)
Steve Haywood had to wait for 0.0 days.
Leslie Cox had to wait for 0.0 days.
Cathryn Shepard had to wait for 0.0 days.
Linda Trigg had to wait for 0.0 days.
30: Shop reorder point hit
Jeffrey Savino had to wait for 0.0 days.
Pauline Johnson had to wait for 0.0 days.
Gabriel Sachez had to wait for 0.0 days.
Florence Trinidad had to wait for 0.0 days.
Carol Harrell had to wait for 0.0 days.
Larraine Maughan had to wait for 0.0 days.
Gale Carson had to wait for 0.0 days.
Mark Serrell had to wait for 0.0 days.
George Thomas had to wait for 0.0 days.
Robert Hernandez had to wait for 0.0 days.
Faith Bermeo had to wait for 0.0 days.
Van Palazzola had to wait for 0.0 days.
Toni Lavallee had to wait for 0.0 days.
Glenn Anderson had to wait for 0.0 days.
Vicki

In [ ]:
print("Minimum ROP threshold that gives wait times of 0.00 for average customer rate of {1} = {0} items".format(current_rop-1, ModelParams.order_rate))

Minimum ROP threshold that gives wait times of 0.00 for average customer rate of 3.2 = 33 items
